все решение выполнялось в гугл колабе на видеокарте T4

In [ ]:
# 1. Обновляем pip и устанавливаем инструменты сборки (это помогает при компиляции)
!pip install --upgrade pip setuptools wheel cmake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.9/28.9 MB 41.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Attempting uninstall: cmake
    Found existing installation: cmake 3.31.10
    Uninstalling cmake-3.31.10:
      Successfully uninstalled cmake-3.31.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


# ПОСЛЕ ЭТОЙ ЯЧЕЙКИ НАДО ПЕРЕЗАПУСТИТЬ КЭРНЕЛ ЕСЛИ ВЫ В КОЛАБЕ

In [ ]:
!pip install Cython

In [2]:
# 2. Устанавливаем библиотеки по очереди (так надежнее)
!pip install catboost
!pip install pandas numpy scikit-learn

# 3. Устанавливаем implicit с флагом verbose, чтобы видеть процесс (если будет компилироваться долго)
# Обычно после установки cmake дело идет быстрее
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 36.1 MB/s  0:00:02
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for implicit: filename=implicit-0.7.2-cp312-cp312-linux_x86_64.whl size=10797912 sha256=805d4057a90c570e2910d0d063ff35e3d7b14c7bef2e0409c1d607fe3e85a91d
  Stored in directory: /root/.cache/pip/wheels/b2/00/4f/9ff8af07a0a53ac6007ea5d739da19cfe147a2df542b6899f8
Successfully built implicit


# ПРЕДЫДУЩАЯ ЯЧЕЙКА МОЖЕТ ВЫПОЛНЯТЬСЯ ДО 10 МИНУТ (В КОЛАБЕ)

In [30]:
import gc
import pickle
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import implicit
from catboost import CatBoostRanker, Pool
from sklearn.model_selection import KFold, train_test_split
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import torch

# --- MASTER CONFIGURATION  ---
class Config:
    RANDOM_STATE = 42
    VAL_SIZE = 0.1

    # Настройки ALS
    ALS_FACTORS = 64
    ALS_REGULARIZATION = 0.01
    ALS_ITERATIONS = 20
    N_FOLDS_ALS = 5

    # Генерация кандидатов
    TOP_K_CANDIDATES = 40     # Hard Negatives от ALS
    NUM_RANDOM_NEGS = 0     # Random Negatives (не нужен)

    # Настройки NLP
    USE_BERT = True        # Включено
    BERT_MODEL = 'deepvk/USER-bge-m3'
    BERT_PCA_COMPONENTS = 32


    USE_TFIDF=True
    TFIDF_MAX_FEATURES=2000
    TFIDF_SVD_COMPONENTS=32



    # Фичи
    USE_META_FEATURES = True
    USE_LOO_STATS = True      # Оставляем Leave-One-Out (безопасно)
    USE_TIME_FEATURES = True

    CAT_COLS = ['author_id', 'publisher', 'language', 'category_id']

CFG = Config()
print("Strict Config loaded.")

# Функция оптимизации памяти (Обязательная)
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object and not str(col_type).startswith('datetime'):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
    print(f'Memory usage reduced to {df.memory_usage().sum() / 1024**2:.2f} MB')
    return df

print("Config loaded.")

Strict Config loaded.
Config loaded.


In [31]:
print("Loading data...")
# Замените пути на свои
train_df = pd.read_csv('train.csv', parse_dates=['timestamp'])
candidates_df = pd.read_csv('candidates.csv')
users_df = pd.read_csv('users.csv')
books_df = pd.read_csv('books.csv')
try:
    print("Попытка загрузить book_descriptions с engine='python'...")
    book_descriptions = pd.read_csv(
        "book_descriptions.csv",
        engine='python',         # Более медленный, но умный движок
        on_bad_lines='skip',     # Пропустить битые строки
        encoding_errors='replace' # Заменить кривые символы
    )
    print(f"Успешно! Загружено строк: {len(book_descriptions)}")
except Exception as e:
    print(f"Ошибка при загрузке описаний: {e}")
    # Создаем пустой датафрейм, чтобы код дальше не падал
    book_descriptions = pd.DataFrame(columns=['book_id', 'description'])

# Оптимизация
train_df = reduce_mem_usage(train_df)
users_df = reduce_mem_usage(users_df)
books_df = reduce_mem_usage(books_df)

# Глобальные маппинги ID (важно для ALS)
unique_users = train_df['user_id'].unique()
unique_items = books_df['book_id'].unique()

user2idx = {u: i for i, u in enumerate(unique_users)}
item2idx = {i: k for k, i in enumerate(unique_items)}
idx2user = {i: u for u, i in user2idx.items()}
idx2item = {k: i for i, k in item2idx.items()}

num_users = len(unique_users)
num_items = len(unique_items)

print(f"Users: {num_users}, Items: {num_items}")

Loading data...
Попытка загрузить book_descriptions с engine='python'...
Успешно! Загружено строк: 55784
Memory usage reduced to 4.62 MB
Memory usage reduced to 0.04 MB
Memory usage reduced to 1.81 MB
Users: 7289, Items: 55784


In [ ]:
def process_text_features(desc_df):
    print("--- Processing Text Features ---")
    desc_df['description'] = desc_df['description'].fillna('')
    final_emb_df = pd.DataFrame({'book_id': desc_df['book_id']})

    # 1. TF-IDF + SVD
    if CFG.USE_TFIDF:
        print("Running TF-IDF + SVD...")
        tfidf = TfidfVectorizer(max_features=CFG.TFIDF_MAX_FEATURES, stop_words='english')
        tfidf_mat = tfidf.fit_transform(desc_df['description'])

        svd = TruncatedSVD(n_components=CFG.TFIDF_SVD_COMPONENTS, random_state=CFG.RANDOM_STATE)
        svd_mat = svd.fit_transform(tfidf_mat)

        cols = [f'tfidf_svd_{i}' for i in range(CFG.TFIDF_SVD_COMPONENTS)]
        temp_df = pd.DataFrame(svd_mat, columns=cols)
        final_emb_df = pd.concat([final_emb_df, temp_df], axis=1)

    # 2. BERT + PCA
    if CFG.USE_BERT:
        print(f"Running BERT ({CFG.BERT_MODEL})...")
        model_bert = SentenceTransformer(CFG.BERT_MODEL)
        # Если есть GPU, используем его
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model_bert.to(device)

        # Получаем эмбеддинги (это может занять время, но rubert-tiny быстрый)
        embeddings = model_bert.encode(
            desc_df['description'].tolist(),
            show_progress_bar=True,
            batch_size=64,
            convert_to_numpy=True
        )

        # Сжимаем PCA
        print("Reducing BERT dimensions with PCA...")
        pca = PCA(n_components=CFG.BERT_PCA_COMPONENTS, random_state=CFG.RANDOM_STATE)
        bert_pca = pca.fit_transform(embeddings)

        cols = [f'bert_pca_{i}' for i in range(CFG.BERT_PCA_COMPONENTS)]
        temp_df = pd.DataFrame(bert_pca, columns=cols)
        final_emb_df = pd.concat([final_emb_df, temp_df], axis=1)

    return reduce_mem_usage(final_emb_df)

# Запускаем обработку текста
if book_descriptions is not None:
    text_features_df = process_text_features(book_descriptions)
else:
    text_features_df = None

генерация эмбедингов этой моделью может длится в районе 40-50 минут (на т4)

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

def create_rich_content_features():
    print("--- Creating Rich Content Features (Genres, Authors, Descs) ---")

    # 1. Загрузка данных (пути могут отличаться)
    books = pd.read_csv('books.csv')
    book_genres = pd.read_csv('book_genres.csv')
    genres = pd.read_csv('genres.csv')

    # === БЛОК 1: ЖАНРЫ (TF-IDF + SVD) ===
    # Жанры - это связь "многие-ко-многим". Схлопнем их в строку.
    print("Processing Genres...")

    # Соединяем ID с названиями
    bg_full = book_genres.merge(genres, on='genre_id', how='left')

    # Группируем жанры для каждой книги в одну строку через пробел
    # Пример: "Фантастика Научная_фантастика Космическая_опера"
    # Заменяем пробелы в названиях жанров на подчеркивания, чтобы TF-IDF не разбил их
    bg_full['genre_name'] = bg_full['genre_name'].fillna('unknown').apply(lambda x: x.replace(' ', '_'))

    genre_strings = bg_full.groupby('book_id')['genre_name'].apply(lambda x: ' '.join(x)).reset_index()
    genre_strings.columns = ['book_id', 'genre_str']

    # TF-IDF на жанрах
    tfidf_g = TfidfVectorizer(token_pattern=r'(?u)\b\w+\b') # Простой токенайзер
    genre_matrix = tfidf_g.fit_transform(genre_strings['genre_str'])

    # SVD сжатие (жанров не так много, 16 компонент хватит за глаза)
    N_GENRE_COMPONENTS = 16
    svd_g = TruncatedSVD(n_components=N_GENRE_COMPONENTS, random_state=42)
    genre_svd = svd_g.fit_transform(genre_matrix)

    # Создаем DataFrame с фичами жанров
    genre_feat_cols = [f'genre_svd_{i}' for i in range(N_GENRE_COMPONENTS)]
    genre_df = pd.DataFrame(genre_svd, columns=genre_feat_cols)
    genre_df['book_id'] = genre_strings['book_id']

    print(f"Genre features shape: {genre_df.shape}")

    # === БЛОК 2: АВТОРЫ И ИЗДАТЕЛЬСТВА (Статистики) ===
    # Вместо SVD по авторам (их слишком много и матрица разрежена),
    # лучше добавить агрегированные статистики.
    print("Processing Authors & Publishers Stats...")

    # Используем train_df для расчета популярности авторов (Data Leakage тут нет, это глобальные тренды)
    # Нам нужно знать глобальный train_df
    train_local = pd.read_csv('train.csv') # Грузим локально для расчетов

    # Мержим с книгами, чтобы получить author_id
    train_w_meta = train_local.merge(books[['book_id', 'author_id', 'publisher']], on='book_id', how='left')

    # Статистики по АВТОРАМ
    author_stats = train_w_meta.groupby('author_id').agg({
        'rating': ['mean', 'count'],
        'has_read': 'mean' # Процент дочитываемости автора
    }).reset_index()
    author_stats.columns = ['author_id', 'author_mean_rating', 'author_book_count', 'author_read_ratio']

    # Статистики по ИЗДАТЕЛЬСТВАМ
    pub_stats = train_w_meta.groupby('publisher').agg({
        'rating': 'mean',
        'has_read': 'mean'
    }).reset_index()
    pub_stats.columns = ['publisher', 'pub_mean_rating', 'pub_read_ratio']

    # === СБОРКА ВСЕГО ВМЕСТЕ ===
    # Берем базовые книги
    final_content_df = books[['book_id', 'author_id', 'publisher', 'publication_year']].copy()

    # Присоединяем жанры (SVD)
    final_content_df = final_content_df.merge(genre_df, on='book_id', how='left')

    # Заполняем пропуски в SVD нулями (для книг без жанров)
    final_content_df[genre_feat_cols] = final_content_df[genre_feat_cols].fillna(0.0)

    # Присоединяем статистику авторов
    final_content_df = final_content_df.merge(author_stats, on='author_id', how='left')

    # Присоединяем статистику издательств
    final_content_df = final_content_df.merge(pub_stats, on='publisher', how='left')

    # Заполняем пропуски в статистиках средними (для новых авторов)
    for col in ['author_mean_rating', 'author_read_ratio', 'pub_mean_rating', 'pub_read_ratio']:
        final_content_df[col] = final_content_df[col].fillna(final_content_df[col].mean())

    final_content_df['author_book_count'] = final_content_df['author_book_count'].fillna(0)

    return reduce_mem_usage(final_content_df)

# ЗАПУСК ГЕНЕРАЦИИ
content_features_v2 = create_rich_content_features()
print("Content Features Ready.")

--- Creating Rich Content Features (Genres, Authors, Descs) ---
Processing Genres...
Genre features shape: (55784, 17)
Processing Authors & Publishers Stats...
Memory usage reduced to 2.98 MB
Content Features Ready.


In [34]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import implicit
from sklearn.model_selection import KFold
import gc
import torch

# Маппинги должны быть уже созданы ранее, но на всякий случай обновим их глобально
unique_users = train_df['user_id'].unique()
unique_items = books_df['book_id'].unique()
user2idx = {u: i for i, u in enumerate(unique_users)}
item2idx = {i: k for k, i in enumerate(unique_items)}
idx2user = {i: u for u, i in user2idx.items()}
idx2item = {k: i for i, k in item2idx.items()}
num_users, num_items = len(unique_users), len(unique_items)

def generate_hybrid_train_data_v8(df, n_hard=40, n_random=10, n_folds=5):
    print(f"--- [v8] Generating Hybrid Train Data (Global Candidates + OOF Scores) ---")

    # === ЭТАП 1: Обучение Глобальной ALS (для выбора кандидатов) ===
    print("1. Training Global ALS (Candidate Selection)...")
    rows = df['user_id'].map(user2idx).values
    cols = df['book_id'].map(item2idx).values
    # Создаем полную матрицу взаимодействий
    sparse_global = sparse.csr_matrix((np.ones(len(rows)), (rows, cols)), shape=(num_users, num_items))

    global_model = implicit.als.AlternatingLeastSquares(
        factors=CFG.ALS_FACTORS,
        regularization=CFG.ALS_REGULARIZATION,
        iterations=CFG.ALS_ITERATIONS,
        random_state=CFG.RANDOM_STATE,
        use_gpu=torch.cuda.is_available()
    )
    global_model.fit(sparse_global, show_progress=True)

    # === ЭТАП 2: Сборка структуры датасета (Строки) ===
    print(f"2. Constructing Dataset Skeleton (Pos + {n_hard} Hard + {n_random} Random)...")

    # А) Позитивы (Target: 2 = Read, 1 = Planned)
    pos_df = df[['user_id', 'book_id', 'has_read']].copy()
    pos_df['target'] = pos_df['has_read'].apply(lambda x: 2.0 if x == 1 else 1.0)
    pos_df.drop(columns=['has_read'], inplace=True)

    # Б) Hard Negatives (Target: 0) - берем топ рекомендаций глобальной модели
    # Это самые "сложные" примеры, где модель ошибается
    all_users_idx = np.arange(num_users)
    ids, _ = global_model.recommend(
        all_users_idx,
        sparse_global[all_users_idx],
        N=n_hard,
        filter_already_liked_items=True
    )

    neg_data = []
    # Разворачиваем рекомендации
    for i, u_idx in enumerate(all_users_idx):
        u_id = idx2user[u_idx]
        for b_idx in ids[i]:
            # Target 0
            neg_data.append([u_id, idx2item[b_idx], 0.0])

    # В) Random Negatives (Target: 0) - для разнообразия и робастности
    rnd_data = []
    if n_random > 0:
        for u_idx in all_users_idx:
            u_id = idx2user[u_idx]
            # Выбираем случайные книги (можно оптимизировать, но цикл надежнее)
            rand_items_idx = np.random.randint(0, num_items, n_random)
            for r_idx in rand_items_idx:
                rnd_data.append([u_id, idx2item[r_idx], 0.0])

    # Сборка
    neg_df = pd.DataFrame(neg_data, columns=['user_id', 'book_id', 'target'])
    rnd_df = pd.DataFrame(rnd_data, columns=['user_id', 'book_id', 'target'])

    full_df = pd.concat([pos_df, neg_df, rnd_df], ignore_index=True)

    # Удаляем дубликаты (Приоритет: Pos > Hard > Random)
    full_df.sort_values(by='target', ascending=False, inplace=True)
    full_df.drop_duplicates(subset=['user_id', 'book_id'], keep='first', inplace=True)

    # Инициализируем колонки для OOF скоров (пока пустые)
    full_df['als_score'] = np.nan
    full_df['als_item_norm'] = np.nan

    # Помечаем реальные позитивы, чтобы знать, что исключать при OOF обучении
    # Создаем set для быстрой проверки: (u_idx, b_idx)
    real_interactions = set(zip(
        pos_df['user_id'].map(user2idx).values,
        pos_df['book_id'].map(item2idx).values
    ))

    print(f"Skeleton created. Rows: {len(full_df)}. Starting OOF Scoring...")

    # === ЭТАП 3: OOF Scoring (Честный расчет скоров) ===
    # Разбиваем ВЕСЬ full_df на фолды. Для каждого фолда обучаем ALS заново,
    # ИСКЛЮЧАЯ из обучения те позитивы, которые попали в валидацию.

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=CFG.RANDOM_STATE)

    # Исходные данные для матрицы (все взаимодействия)
    train_u_indices = df['user_id'].map(user2idx).values
    train_b_indices = df['book_id'].map(item2idx).values

    fold_cnt = 0
    for _, val_idx in kf.split(full_df):
        fold_cnt += 1

        # Строки, которые мы сейчас будем скорить
        current_rows = full_df.iloc[val_idx]

        # Находим среди них те, которые являются РЕАЛЬНЫМИ позитивами
        # Их нужно "спрятать" от ALS
        current_u_idxs = current_rows['user_id'].map(user2idx).fillna(-1).astype(int).values
        current_b_idxs = current_rows['book_id'].map(item2idx).fillna(-1).astype(int).values

        # Собираем пары (u, i), которые надо исключить из обучения
        pairs_to_exclude = set()
        for u, b in zip(current_u_idxs, current_b_idxs):
            if (u, b) in real_interactions:
                pairs_to_exclude.add((u, b))

        # Строим обучающую матрицу для этого фолда
        # Фильтруем исходный train_df
        mask_keep = []
        for u, b in zip(train_u_indices, train_b_indices):
            if (u, b) in pairs_to_exclude:
                mask_keep.append(False)
            else:
                mask_keep.append(True)

        mask_keep = np.array(mask_keep)

        # Создаем разреженную матрицу (без валидационных позитивов)
        sparse_fold = sparse.csr_matrix(
            (np.ones(mask_keep.sum()), (train_u_indices[mask_keep], train_b_indices[mask_keep])),
            shape=(num_users, num_items)
        )

        # Обучаем ALS для фолда
        model_fold = implicit.als.AlternatingLeastSquares(
            factors=CFG.ALS_FACTORS, regularization=CFG.ALS_REGULARIZATION,
            iterations=CFG.ALS_ITERATIONS, random_state=CFG.RANDOM_STATE,
            use_gpu=torch.cuda.is_available()
        )
        model_fold.fit(sparse_fold, show_progress=False)

        # Предсказываем (Dot Product)
        if hasattr(model_fold.user_factors, "to_numpy"):
            u_fact = model_fold.user_factors.to_numpy()
            i_fact = model_fold.item_factors.to_numpy()
        else:
            u_fact = model_fold.user_factors
            i_fact = model_fold.item_factors

        # Берем вектора для текущих юзеров и книг (даже если юзер был удален из трейна, вектор будет, хоть и плохой)
        # Обработка -1 не нужна, так как skeleton построен на известных ID
        vectors_u = u_fact[current_u_idxs]
        vectors_b = i_fact[current_b_idxs]

        scores = (vectors_u * vectors_b).sum(axis=1)
        norms = np.linalg.norm(vectors_b, axis=1)

        # Записываем
        full_df.iloc[val_idx, full_df.columns.get_loc('als_score')] = scores
        full_df.iloc[val_idx, full_df.columns.get_loc('als_item_norm')] = norms

        print(f"Fold {fold_cnt}/{n_folds} scored.")

        # Чистим память
        del model_fold, sparse_fold, vectors_u, vectors_b
        gc.collect()

    full_df = full_df.sample(frac=1, random_state=CFG.RANDOM_STATE).reset_index(drop=True)
    print(f"Hybrid Dataset v8 Ready.\nTarget Counts:\n{full_df['target'].value_counts()}")

    return reduce_mem_usage(full_df)

In [35]:
# Предварительный расчет ГЛОБАЛЬНЫХ статистик (один раз на весь ноутбук)
# Добавляем "дочитываемость" (read_ratio) - сильная фича из v4
global_book_stats = train_df.groupby('book_id').agg({
    'rating': ['mean', 'count'],
    'has_read': ['mean', 'sum']
})
global_book_stats.columns = ['book_mean', 'book_count', 'book_read_ratio', 'book_read_sum']

global_user_stats = train_df.groupby('user_id').agg({
    'rating': ['mean', 'count'],
    'has_read': ['mean', 'sum']
})
global_user_stats.columns = ['user_mean', 'user_count', 'user_read_ratio', 'user_read_sum']

# Глобальные средние для заполнения пропусков
GLOBAL_MEAN_RATING = train_df['rating'].mean()
GLOBAL_MEAN_READ = train_df['has_read'].mean()

def feature_engineering_v8(df):
    print(f"--- Feature Engineering v8 (Enhanced with Genres & Stats) ---")
    out_df = df.copy()

    # 1. Merging Content Features (Metadata)
    # Проверяем, есть ли расширенные фичи (с жанрами и статистикой авторов)
    if 'content_features_v2' in globals() and content_features_v2 is not None:
        print("Using Rich Content Features (Genres + Author Stats)...")
        out_df = out_df.merge(content_features_v2, on='book_id', how='left')
    else:
        print("WARNING: 'content_features_v2' not found. Using basic books.csv.")
        out_df = out_df.merge(books_df, on='book_id', how='left')

    # Мержим данные пользователей
    out_df = out_df.merge(users_df, on='user_id', how='left')

    # Обработка категориальных признаков (ID должны быть int)
    for c in CFG.CAT_COLS:
        if c in out_df.columns:
            out_df[c] = out_df[c].fillna(-1).astype(int)

    # 2. Text Features (BERT/TF-IDF)
    # Используем уже подготовленный text_features_df (из описаний книг)
    if 'text_features_df' in globals() and text_features_df is not None:
        out_df = out_df.merge(text_features_df, on='book_id', how='left')
        feat_cols = [c for c in text_features_df.columns if c != 'book_id']
        out_df[feat_cols] = out_df[feat_cols].fillna(0.0)

    # 3. Statistics (Global Merge)
    # Мержим глобальные статистики по взаимодействиям
    out_df = out_df.merge(global_book_stats, on='book_id', how='left')
    out_df = out_df.merge(global_user_stats, on='user_id', how='left')

    # Заполнение пропусков глобальными средними
    out_df['book_mean'] = out_df['book_mean'].fillna(GLOBAL_MEAN_RATING)
    out_df['book_count'] = out_df['book_count'].fillna(0)
    out_df['book_read_ratio'] = out_df['book_read_ratio'].fillna(GLOBAL_MEAN_READ)
    out_df['book_read_sum'] = out_df['book_read_sum'].fillna(0)

    out_df['user_mean'] = out_df['user_mean'].fillna(GLOBAL_MEAN_RATING)
    out_df['user_count'] = out_df['user_count'].fillna(0)
    out_df['user_read_ratio'] = out_df['user_read_ratio'].fillna(GLOBAL_MEAN_READ)
    out_df['user_read_sum'] = out_df['user_read_sum'].fillna(0)

    # 4. Time Features
    if CFG.USE_TIME_FEATURES:
        cur_year = 2025
        if 'publication_year' in out_df.columns:
            out_df['publication_year'] = out_df['publication_year'].fillna(cur_year)
            out_df['book_age'] = cur_year - out_df['publication_year']
            # Клипаем выбросы (книги старше 200 лет считаем просто "старыми")
            out_df['book_age'] = out_df['book_age'].clip(0, 200)

    # 5. Чистка лишнего
    # Удаляем текстовые поля и временные метки, чтобы модель не переобучалась на мусоре
    drops = ['title', 'author_name', 'description', 'timestamp', 'has_read',
             'genre_str', 'genre_name', 'genre_string'] # Добавил возможные артефакты жанров

    out_df.drop([c for c in drops if c in out_df.columns], axis=1, inplace=True)

    return reduce_mem_usage(out_df)

In [36]:
# 1. Генерируем "честный" обучающий датасет (включая позитивы и негативы)
train_dataset = generate_hybrid_train_data_v8(train_df, n_hard=CFG.TOP_K_CANDIDATES, n_random=CFG.NUM_RANDOM_NEGS, n_folds=CFG.N_FOLDS_ALS)
# 2. Навешиваем фичи (текстовые + статистики с LOO)
# mode='train' включит защиту от лика таргета в статистиках
X_train_strict = feature_engineering_v8(train_dataset)

print("\nReady for CatBoost Training.")
print("Train Shape:", X_train_strict.shape)
print("Columns:", list(X_train_strict.columns))

--- [v8] Generating Hybrid Train Data (Global Candidates + OOF Scores) ---
1. Training Global ALS (Candidate Selection)...


  0%|          | 0/20 [00:00<?, ?it/s]

2. Constructing Dataset Skeleton (Pos + 40 Hard + 0 Random)...


/tmp/ipython-input-401261207.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat([pos_df, neg_df, rnd_df], ignore_index=True)


Skeleton created. Rows: 560621. Starting OOF Scoring...
Fold 1/5 scored.
Fold 2/5 scored.
Fold 3/5 scored.
Fold 4/5 scored.
Fold 5/5 scored.
Hybrid Dataset v8 Ready.
Target Counts:
target
0.0    291560
2.0    156603
1.0    112458
Name: count, dtype: int64
Memory usage reduced to 11.76 MB
--- Feature Engineering v8 (Enhanced with Genres & Stats) ---
Using Rich Content Features (Genres + Author Stats)...
Memory usage reduced to 117.63 MB

Ready for CatBoost Training.
Train Shape: (560638, 104)
Columns: ['user_id', 'book_id', 'target', 'als_score', 'als_item_norm', 'author_id', 'publisher', 'publication_year', 'genre_svd_0', 'genre_svd_1', 'genre_svd_2', 'genre_svd_3', 'genre_svd_4', 'genre_svd_5', 'genre_svd_6', 'genre_svd_7', 'genre_svd_8', 'genre_svd_9', 'genre_svd_10', 'genre_svd_11', 'genre_svd_12', 'genre_svd_13', 'genre_svd_14', 'genre_svd_15', 'author_mean_rating', 'author_book_count', 'author_read_ratio', 'pub_mean_rating', 'pub_read_ratio', 'gender', 'age', 'tfidf_svd_0', 'tfidf

In [37]:
# --- CELL 1: SETUP, INSTALLS & DATA PREPARATION ---
import gc
import pickle
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRanker, Pool
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch

# 1. Установка библиотек (если нет)
!pip install lightgbm xgboost
# 1. Генерация данных
train_dataset_v8 = generate_hybrid_train_data_v8(train_df, n_hard=CFG.TOP_K_CANDIDATES, n_random=CFG.NUM_RANDOM_NEGS, n_folds=CFG.N_FOLDS_ALS)

# 2. Генерация фичей
X_train_v8 = feature_engineering_v8(train_dataset_v8)

print("\n--- Training CatBoost Ranker (v9) ---")
# Сортировка для групповой обработки
X_train_v8.sort_values(by='user_id', inplace=True)
# 2. Очистка памяти перед стартом
print("--- Initial Memory Cleaning ---")
gc.collect()
torch.cuda.empty_cache()

# 3. Подготовка данных
# Используем датасет, который ты создал ранее: X_train_v8
# Важно: Сортировка по user_id ОБЯЗАТЕЛЬНА для задач ранжирования (LGBM/XGB требуют, чтобы группы шли подряд)
print("Sorting data by user_id...")
df_work = X_train_v8.copy()
df_work.sort_values(by='user_id', inplace=True)

# Определяем колонки
drop_cols = ['user_id', 'book_id', 'target']
feature_cols = [c for c in df_work.columns if c not in drop_cols]
cat_cols = [c for c in CFG.CAT_COLS if c in feature_cols]

print(f"Features ({len(feature_cols)}): {feature_cols}")

# 4. Кодирование категориальных признаков (Label Encoding)
# Это нужно для XGBoost и LightGBM, так как они не принимают строки напрямую
print("Encoding categorical features...")
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    # Заполняем пропуски строкой, чтобы энкодер не падал
    df_work[col] = df_work[col].fillna("Unknown") if df_work[col].dtype == 'object' else df_work[col].fillna(-1)
    # Приводим к строке для унификации перед кодированием
    df_work[col] = df_work[col].astype(str)
    df_work[col] = le.fit_transform(df_work[col])
    label_encoders[col] = le

# 5. Разделение на Train / Val по пользователям
# Нельзя разрывать группы одного юзера
unique_users = df_work['user_id'].unique()
train_users, val_users = train_test_split(unique_users, test_size=CFG.VAL_SIZE, random_state=CFG.RANDOM_STATE)

train_df = df_work[df_work['user_id'].isin(train_users)]
val_df = df_work[df_work['user_id'].isin(val_users)]

# 6. Подготовка групп (Query Groups) для LGBM и XGBoost
# Нам нужно знать, сколько айтемов у каждого юзера подряд
print("Calculating group sizes...")
train_group_sizes = train_df.groupby('user_id', sort=False).size().values
val_group_sizes = val_df.groupby('user_id', sort=False).size().values

# Подготовка X и y
X_train = train_df[feature_cols]
y_train = train_df['target']
X_val = val_df[feature_cols]
y_val = val_df['target']

# Отдельно сохраняем group_id для CatBoost (ему нужен просто столбец)
train_group_id = train_df['user_id']
val_group_id = val_df['user_id']

# Чистим лишнее
del df_work
gc.collect()

print(f"Data Prepared. Train shape: {X_train.shape}, Val shape: {X_val.shape}")
print(f"Train Groups: {len(train_group_sizes)}, Val Groups: {len(val_group_sizes)}")

--- [v8] Generating Hybrid Train Data (Global Candidates + OOF Scores) ---
1. Training Global ALS (Candidate Selection)...


  0%|          | 0/20 [00:00<?, ?it/s]

2. Constructing Dataset Skeleton (Pos + 40 Hard + 0 Random)...


/tmp/ipython-input-401261207.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat([pos_df, neg_df, rnd_df], ignore_index=True)


Skeleton created. Rows: 560621. Starting OOF Scoring...
Fold 1/5 scored.
Fold 2/5 scored.
Fold 3/5 scored.
Fold 4/5 scored.
Fold 5/5 scored.
Hybrid Dataset v8 Ready.
Target Counts:
target
0.0    291560
2.0    156603
1.0    112458
Name: count, dtype: int64
Memory usage reduced to 11.76 MB
--- Feature Engineering v8 (Enhanced with Genres & Stats) ---
Using Rich Content Features (Genres + Author Stats)...
Memory usage reduced to 117.63 MB

--- Training CatBoost Ranker (v9) ---
--- Initial Memory Cleaning ---
Sorting data by user_id...
Features (101): ['als_score', 'als_item_norm', 'author_id', 'publisher', 'publication_year', 'genre_svd_0', 'genre_svd_1', 'genre_svd_2', 'genre_svd_3', 'genre_svd_4', 'genre_svd_5', 'genre_svd_6', 'genre_svd_7', 'genre_svd_8', 'genre_svd_9', 'genre_svd_10', 'genre_svd_11', 'genre_svd_12', 'genre_svd_13', 'genre_svd_14', 'genre_svd_15', 'author_mean_rating', 'author_book_count', 'author_read_ratio', 'pub_mean_rating', 'pub_read_ratio', 'gender', 'age', 'tfid

In [39]:
# --- CELL 2: ENSEMBLE TRAINING LOOP (TUNED VERSION) ---

# Оптимизированные параметры
MODELS_CONFIG = [
    {
        'name': 'catboost_yeti',
        'type': 'catboost',
        'params': {
            'loss_function': 'YetiRank',
            'iterations': 1500,
            'learning_rate':   0.2192448884902637,
            'depth': 6,
            'l2_leaf_reg':5.7,
            'task_type': 'GPU' if torch.cuda.is_available() else 'CPU',
            'random_seed': CFG.RANDOM_STATE,
            'eval_metric': 'NDCG:top=20',
            'verbose': 100,
            'early_stopping_rounds': 100
        }
    },
    {
        'name': 'catboost_pairlogit',
        'type': 'catboost',
        'params': {
            'loss_function': 'PairLogitPairwise',
            'iterations': 1200,
            'learning_rate': 0.08,
            'depth': 8,
            'l2_leaf_reg': 5,
            'task_type': 'GPU' if torch.cuda.is_available() else 'CPU',
            'random_seed': CFG.RANDOM_STATE,
            'eval_metric': 'NDCG:top=20',
            'metric_period': 5,
            'verbose': 50,
            'early_stopping_rounds': 100
        }
    },
    {
        'name': 'lgbm_lambdarank',
        'type': 'lgbm',
        'params': {
            'objective': 'lambdarank',
            'metric': 'ndcg',
            'eval_at': [20],
            'learning_rate': 0.07,
            'n_estimators': 3000,
            'max_depth': 7,
            'num_leaves': 91,
            'colsample_bytree': 0.725,
            'subsample': 0.8,
            'min_child_samples': 33,
            'random_state': CFG.RANDOM_STATE,
            'verbosity': -1,
            'device': 'cpu' if torch.cuda.is_available() else 'cpu'
        }
    },
    {
        'name': 'xgboost_pairwise',
        'type': 'xgboost',
        'params': {
            'objective': 'rank:pairwise',
            'eval_metric': 'ndcg@20',
            'learning_rate': 0.18,
            'n_estimators': 3000,
            'max_depth': 6,
            'subsample': 0.77,
            'colsample_bytree': 0.62,
            'gamma': 1.0,
            'min_child_weight': 5,
            'tree_method': 'hist',
            'device': 'cuda' if torch.cuda.is_available() else 'cpu',
            'random_state': CFG.RANDOM_STATE
        }
    }
]

trained_models = {}

for config in MODELS_CONFIG:
    model_name = config['name']
    model_type = config['type']
    params = config['params']

    print(f"\n{'='*20} Training {model_name} ({model_type}) {'='*20}")

    if model_type == 'catboost':
        train_pool = Pool(X_train, y_train, group_id=train_group_id, cat_features=cat_cols)
        val_pool = Pool(X_val, y_val, group_id=val_group_id, cat_features=cat_cols)

        model = CatBoostRanker(**params)
        model.fit(train_pool, eval_set=val_pool)

        trained_models[model_name] = model
        print(f"Best Score: {model.get_best_score()}")
        del train_pool, val_pool

    elif model_type == 'lgbm':
        # !!! ВАЖНО: Явно передаем индексы категорий
        cat_indices = [X_train.columns.get_loc(c) for c in cat_cols]
        print(f"Passing {len(cat_indices)} categorical features to LGBM...")

        # Передаем индексы категорий при создании Dataset
        dtrain = lgb.Dataset(X_train, label=y_train, group=train_group_sizes, categorical_feature=cat_indices)
        dval = lgb.Dataset(X_val, label=y_val, group=val_group_sizes, reference=dtrain, categorical_feature=cat_indices)

        model = lgb.train(
            params,
            dtrain,
            valid_sets=[dval],
            callbacks=[lgb.early_stopping(stopping_rounds=150), lgb.log_evaluation(200)]
            # Отсюда аргумент categorical_feature убрали
        )

        trained_models[model_name] = model
        del dtrain, dval

    elif model_type == 'xgboost':
        dtrain = xgb.DMatrix(X_train, y_train)
        dtrain.set_group(train_group_sizes)

        dval = xgb.DMatrix(X_val, y_val)
        dval.set_group(val_group_sizes)

        model = xgb.train(
            params,
            dtrain,
            num_boost_round=params['n_estimators'],
            evals=[(dtrain, 'train'), (dval, 'valid')],
            early_stopping_rounds=150, # Увеличили терпение
            verbose_eval=200
        )

        trained_models[model_name] = model
        del dtrain, dval

    # Очистка памяти
    gc.collect()
    torch.cuda.empty_cache()
    print(f"Finished {model_name}. Memory cleaned.")

print("\nAll models trained successfully!")


==================== Training catboost_yeti (catboost) ====================
Groupwise loss function. OneHotMaxSize set to 10


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=20;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.7544952	best: 0.7544952 (0)	total: 117ms	remaining: 2m 54s
100:	test: 0.8894124	best: 0.8895586 (97)	total: 6.91s	remaining: 1m 35s
200:	test: 0.8959444	best: 0.8960115 (199)	total: 13.4s	remaining: 1m 26s
300:	test: 0.9000355	best: 0.9000519 (299)	total: 20.4s	remaining: 1m 21s
400:	test: 0.9006124	best: 0.9007113 (397)	total: 26.9s	remaining: 1m 13s
500:	test: 0.9027938	best: 0.9033273 (497)	total: 33s	remaining: 1m 5s
600:	test: 0.9038371	best: 0.9038630 (599)	total: 39.7s	remaining: 59.4s
700:	test: 0.9046712	best: 0.9046722 (698)	total: 45.6s	remaining: 52s
800:	test: 0.9046543	best: 0.9048384 (741)	total: 52.2s	remaining: 45.5s
900:	test: 0.9050124	best: 0.9050782 (835)	total: 58.1s	remaining: 38.6s
1000:	test: 0.9052322	best: 0.9053601 (992)	total: 1m 4s	remaining: 32.2s
1100:	test: 0.9054583	best: 0.9056199 (1062)	total: 1m 10s	remaining: 25.6s
bestTest = 0.9056199216
bestIteration = 1062
Shrink model to first 1063 iterations.
Best Score: {}
Finished catboost_yeti. M

Metric NDCG:top=20;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.7935516	best: 0.7935516 (0)	total: 1.12s	remaining: 22m 27s
50:	test: 0.8905744	best: 0.8905744 (50)	total: 54.7s	remaining: 20m 32s
100:	test: 0.8957470	best: 0.8962133 (97)	total: 1m 48s	remaining: 19m 35s
150:	test: 0.8986981	best: 0.8986981 (150)	total: 2m 42s	remaining: 18m 49s
200:	test: 0.9011359	best: 0.9014531 (199)	total: 3m 36s	remaining: 17m 57s
250:	test: 0.9022394	best: 0.9024029 (234)	total: 4m 31s	remaining: 17m 5s
300:	test: 0.9022058	best: 0.9026235 (297)	total: 5m 25s	remaining: 16m 11s
350:	test: 0.9025578	best: 0.9026889 (345)	total: 6m 19s	remaining: 15m 17s
400:	test: 0.9031613	best: 0.9035811 (369)	total: 7m 12s	remaining: 14m 22s
450:	test: 0.9042164	best: 0.9042164 (450)	total: 8m 6s	remaining: 13m 27s
500:	test: 0.9045044	best: 0.9046753 (480)	total: 9m	remaining: 12m 33s
550:	test: 0.9054029	best: 0.9058966 (538)	total: 9m 53s	remaining: 11m 39s
600:	test: 0.9050599	best: 0.9058966 (538)	total: 10m 47s	remaining: 10m 45s
bestTest = 0.9058965556
be

/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [19:22:09] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()


[0]	train-ndcg@20:0.82777	valid-ndcg@20:0.83285
[200]	train-ndcg@20:0.90575	valid-ndcg@20:0.88853
[337]	train-ndcg@20:0.91554	valid-ndcg@20:0.88691
Finished xgboost_pairwise. Memory cleaned.

All models trained successfully!


In [40]:
# --- CELL 3: INFERENCE, BLENDING & SUBMISSION ---
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import implicit
import gc

BLEND_WEIGHTS = {
    'catboost_yeti': 0.5,
    'catboost_pairlogit': 0.3,
    'lgbm_lambdarank': 0.1,
    'xgboost_pairwise': 0.1
}

print("--- [Final] Preparing Test Data ---")

# 1. Сборка тестового датафрейма
candidates_exploded = []
candidates_source = candidates_df.copy() # Из загруженного в начале файла

for _, row in candidates_source.iterrows():
    u_id = row['user_id']
    if pd.isna(row['book_id_list']): continue
    b_ids = str(row['book_id_list']).split(',')
    for b_id in b_ids:
        if b_id: candidates_exploded.append([u_id, int(b_id)])

test_df = pd.DataFrame(candidates_exploded, columns=['user_id', 'book_id'])
print(f"Test pairs generated: {len(test_df)}")

# 2. Пересчет ALS score для теста
print("Recalculating ALS scores for test...")

# Строим матрицу на трейне
train_full_local = pd.read_csv('train.csv')
u_idx_arr = train_full_local['user_id'].map(user2idx).values
i_idx_arr = train_full_local['book_id'].map(item2idx).values
sparse_M = sparse.csr_matrix((np.ones(len(u_idx_arr)), (u_idx_arr, i_idx_arr)), shape=(num_users, num_items))

# Обучаем быструю ALS
model_als_test = implicit.als.AlternatingLeastSquares(
    factors=CFG.ALS_FACTORS,
    iterations=15, # Чуть больше итераций для точности
    random_state=CFG.RANDOM_STATE,
    use_gpu=torch.cuda.is_available()
)
model_als_test.fit(sparse_M, show_progress=False)

# Предсказываем (Dot Product)
u_test_idx = test_df['user_id'].map(user2idx).fillna(-1).astype(int).values
b_test_idx = test_df['book_id'].map(item2idx).fillna(-1).astype(int).values
mask = (u_test_idx >= 0) & (b_test_idx >= 0)

scores = np.zeros(len(test_df))
norms = np.zeros(len(test_df))

# --- ИСПРАВЛЕНИЕ ОШИБКИ ЗДЕСЬ (if -> itf) ---
if hasattr(model_als_test.user_factors, "to_numpy"):
    uf = model_als_test.user_factors.to_numpy()
    itf = model_als_test.item_factors.to_numpy() # Исправлено имя переменной
else:
    uf = model_als_test.user_factors
    itf = model_als_test.item_factors # Исправлено имя переменной

scores[mask] = (uf[u_test_idx[mask]] * itf[b_test_idx[mask]]).sum(axis=1)
norms[mask] = np.linalg.norm(itf[b_test_idx[mask]], axis=1)

test_df['als_score'] = scores
test_df['als_item_norm'] = norms

# Чистим память от ALS
del model_als_test, sparse_M, uf, itf
gc.collect()

# 3. Генерация фичей
print("Generating features for test...")
test_df = feature_engineering_v8(test_df)

# 4. Применение LabelEncoder к тесту (для LGBM/XGB)
print("Applying encodings to test...")
for col in cat_cols:
    le = label_encoders.get(col)
    if le:
        test_df[col] = test_df[col].fillna("Unknown").astype(str)
        known_classes = set(le.classes_)
        # Заменяем неизвестные категории на самый частый класс (обычно 0) или "Unknown" если есть
        fallback_val = le.classes_[0]
        test_df[col] = test_df[col].apply(lambda x: x if x in known_classes else fallback_val)
        test_df[col] = le.transform(test_df[col])

# Выравнивание колонок
X_test = test_df[feature_cols].copy()

# 5. Цикл предсказаний
print("--- Predicting with Ensemble ---")
preds_df = test_df[['user_id', 'book_id']].copy()

for name, model in trained_models.items():
    print(f"Predicting: {name}...")

    try:
        if 'catboost' in name:
            # CatBoost Pool
            test_pool = Pool(X_test, group_id=test_df['user_id'], cat_features=cat_cols)
            raw_preds = model.predict(test_pool)

        elif 'lgbm' in name:
            raw_preds = model.predict(X_test)

        elif 'xgboost' in name:
            dtest = xgb.DMatrix(X_test)
            raw_preds = model.predict(dtest)

        preds_df[f'score_{name}'] = raw_preds
    except Exception as e:
        print(f"Error predicting with {name}: {e}")

# 6. Статистика и Блендинг
score_cols = [c for c in preds_df.columns if c.startswith('score_')]
print("\n--- Model Correlations ---")
print(preds_df[score_cols].corr())

print("\n--- Predictions Statistics (Raw) ---")
print(preds_df[score_cols].describe())

# Rank Normalization
print("Normalizing ranks...")
for col in score_cols:
    preds_df[f'rank_{col}'] = preds_df.groupby('user_id')[col].rank(pct=True)

# Взвешенный блендинг
print("Blending...")
preds_df['final_blend_score'] = 0
total_weight = 0

for name, weight in BLEND_WEIGHTS.items():
    col_name = f'score_{name}'
    # Проверяем, есть ли предсказания этой модели (вдруг обучение упало)
    if col_name in preds_df.columns:
        preds_df['final_blend_score'] += preds_df[f'rank_{col_name}'] * weight
        total_weight += weight
    else:
        print(f"Warning: {name} predictions not found, skipping in blend.")

if total_weight > 0:
    preds_df['final_blend_score'] /= total_weight
else:
    print("Error: No models available for blending!")

# 7. Сохранение
def save_submission(df, score_col, filename):
    print(f"Generating {filename}...")
    # Сортируем: сначала User, потом Score (descending)
    df_sorted = df.sort_values(['user_id', score_col], ascending=[True, False])
    # Убираем дубли книг
    df_sorted = df_sorted.drop_duplicates(subset=['user_id', 'book_id'])
    # Группируем
    submission = df_sorted.groupby('user_id')['book_id'].apply(list).reset_index()
    # Форматируем топ-20
    submission['book_id_list'] = submission['book_id'].apply(lambda x: ",".join([str(i) for i in x[:20]]))
    # Сохраняем (мердж с candidates_source, чтобы сохранить всех юзеров)
    final_sub = candidates_source[['user_id']].merge(submission[['user_id', 'book_id_list']], on='user_id', how='left')
    final_sub.to_csv(filename, index=False)
    print(f"Saved to {filename}")

# Сохраняем индивидуальные сабмиты
for col in score_cols:
    model_name = col.replace('score_', '')
    save_submission(preds_df, col, f'submission_{model_name}.csv')

# Сохраняем Блендинг
if 'final_blend_score' in preds_df.columns:
    save_submission(preds_df, 'final_blend_score', 'submission_ENSEMBLE_BLENDED.csv')

print("\nDONE! All submissions saved.")
print(preds_df.head())

--- [Final] Preparing Test Data ---
Test pairs generated: 81048
Recalculating ALS scores for test...
Generating features for test...
--- Feature Engineering v8 (Enhanced with Genres & Stats) ---
Using Rich Content Features (Genres + Author Stats)...
Memory usage reduced to 16.23 MB
Applying encodings to test...
--- Predicting with Ensemble ---
Predicting: catboost_yeti...
Predicting: catboost_pairlogit...
Predicting: lgbm_lambdarank...
Predicting: xgboost_pairwise...

--- Model Correlations ---
                          score_catboost_yeti  score_catboost_pairlogit  \
score_catboost_yeti                  1.000000                  0.970146   
score_catboost_pairlogit             0.970146                  1.000000   
score_lgbm_lambdarank                0.968597                  0.969559   
score_xgboost_pairwise               0.954572                  0.965846   

                          score_lgbm_lambdarank  score_xgboost_pairwise  
score_catboost_yeti                    0.968597   

В качестве финального файла отправки использовать submission_ENSEMBLE_BLENDED.csv

Все про ии и решение я напишу в файле в корне гита